# Kubernetes Replication

You almost never create pods directly. Instead, you create other types of resources, such as Replication-Controllers or Deployments, which then create and manage the actual pods.

### 1. Replication Controllers (deprecated)

If a pod disappears for any reason, such as in case of a node failure, the ReplicationController notices the missing pod and creates a replacement pod.

A ReplicationController constantly monitors the list of running pods and makes sure the actual number of pods of a “type” always matches the desired number. If too few such pods are running, it creates new replicas from a pod template. If too many such pods are running, it removes the excess replicas.

**Create replication controller**: yaml file: rc_node.yaml

Note: pod template in rc yaml file need to have labels, so that the rc knows which pods to track.

In [2]:
%%bash
kubectl apply -f rc_node.yaml

replicationcontroller/nodeapp-rc created


In [3]:
%%bash
kubectl get pods

NAME               READY   STATUS    RESTARTS   AGE
nodeapp-rc-j7vs8   1/1     Running   0          13s
nodeapp-rc-jznm2   1/1     Running   0          13s


**-> Two pods running as specified in the yaml file**

In [4]:
%%bash
kubectl get rc

NAME         DESIRED   CURRENT   READY   AGE
nodeapp-rc   2         2         2       3m53s


Note: Changing a ReplicationController’s pod template only affects pods created afterward and has no effect on existing pods

Note: Which pods are controlled by an RC is defined via labels. If you remove the respective labels from a pod, its no longer controlled...

**Delete RC**

In [5]:
%%bash
kubectl delete rc nodeapp-rc

replicationcontroller "nodeapp-rc" deleted


In [7]:
%%bash
kubectl get pods

No resources found in default namespace.


### 2. Replica Sets

**Replica Sets are the new version of Replication Controllers**

Whereas a ReplicationController’s label selector only allows matching pods that include a certain label, a ReplicaSet’s selector also allows matching pods that lack a certain label or pods that include a certain label key, regardless of its value.

**Create ReplicaSet from file:** rs_node.yaml (same as with rc with simple label selector "matchLabels")

In [8]:
%%bash
kubectl apply -f rs_node.yaml

replicaset.apps/nodeapp-rs created


In [9]:
%%bash
kubectl get pods

NAME               READY   STATUS    RESTARTS   AGE
nodeapp-rs-mw4lm   1/1     Running   0          11s
nodeapp-rs-x2g47   1/1     Running   0          11s


In [10]:
%%bash
kubectl delete rs nodeapp-rs

replicaset.apps "nodeapp-rs" deleted


**Note**: You can add more complex label selectors in ReplicaSets

### 3. Daemon Sets

With Daemon Sets you can insure that one instance of a pod is running on every node. They are used for infrastructure services like log collection.

In DaemonSets no replica counts are specified, otherwise they are created like Replica Sets

### 4. Jobs

ReplicationControllers, ReplicaSets, and DaemonSets run continuous tasks that are never considered completed. If such pods exit, they are automatically restarted. Jobs on the other hand should not restart after completion.

**Simulated Python Batch Job**: simple-batch-job that completes in 10 sec and exits with error in 1/10 cases by random -> Create image and upload to registry

In [12]:
%%bash
docker build -t ardconsulting/test:py-batch simple-batch-job

Sending build context to Docker daemon  3.072kB
Step 1/4 : FROM python:3
3: Pulling from library/python
e9afc4f90ab0: Pulling fs layer
989e6b19a265: Pulling fs layer
af14b6c2f878: Pulling fs layer
5573c4b30949: Pulling fs layer
11a88e764313: Pulling fs layer
ee776f0e36af: Pulling fs layer
513c90a1afc3: Pulling fs layer
df9b9e95bdb9: Pulling fs layer
86c9edb54464: Pulling fs layer
11a88e764313: Waiting
ee776f0e36af: Waiting
513c90a1afc3: Waiting
df9b9e95bdb9: Waiting
86c9edb54464: Waiting
5573c4b30949: Waiting
af14b6c2f878: Download complete
989e6b19a265: Verifying Checksum
989e6b19a265: Download complete
e9afc4f90ab0: Verifying Checksum
e9afc4f90ab0: Download complete
ee776f0e36af: Verifying Checksum
ee776f0e36af: Download complete
e9afc4f90ab0: Pull complete
989e6b19a265: Pull complete
af14b6c2f878: Pull complete
513c90a1afc3: Verifying Checksum
513c90a1afc3: Download complete
df9b9e95bdb9: Verifying Checksum
df9b9e95bdb9: Download complete
86c9edb54464: Verifying Checksum
86c9edb5446

In [13]:
%%bash
docker push ardconsulting/test:py-batch

The push refers to repository [docker.io/ardconsulting/test]
88f21c9b0327: Preparing
001536999667: Preparing
ccbefb30278f: Preparing
7a8a38bf5538: Preparing
0d77d4546954: Preparing
98d95bdfa037: Preparing
da9418a2e1b1: Preparing
2e5b4ca91984: Preparing
527ade4639e0: Preparing
c2c789d2d3c5: Preparing
8803ef42039d: Preparing
da9418a2e1b1: Waiting
2e5b4ca91984: Waiting
527ade4639e0: Waiting
c2c789d2d3c5: Waiting
8803ef42039d: Waiting
98d95bdfa037: Waiting
0d77d4546954: Mounted from library/python
ccbefb30278f: Mounted from library/python
7a8a38bf5538: Mounted from library/python
001536999667: Pushed
88f21c9b0327: Pushed
da9418a2e1b1: Mounted from library/python
98d95bdfa037: Mounted from library/python
2e5b4ca91984: Mounted from library/python
c2c789d2d3c5: Mounted from library/python
8803ef42039d: Mounted from library/python
527ade4639e0: Pushed
py-batch: digest: sha256:1a4155785be852233c30cdbcd603dc52752d0cdbd9377e2cac68cf3147ee5dae size: 2631


**yaml File for batch job**: simple_job.yaml

In [14]:
%%bash
kubectl apply -f simple_job.yaml

job.batch/batch-job created


In [15]:
%%bash
kubectl get jobs

NAME        COMPLETIONS   DURATION   AGE
batch-job   1/1           8s         57s


In [23]:
%%bash
kubectl get pods

NAME              READY   STATUS      RESTARTS   AGE
batch-job-nv5xm   0/1     Completed   0          4m27s


Show logs:

In [24]:
%%bash
kubectl logs batch-job-nv5xm

Batch Job started
Batch Job completed successfully


In [25]:
%%bash
kubectl delete job batch-job

job.batch "batch-job" deleted


### 5. CronJobs

**Run job every minute**: simple_cronjob.yaml

In [28]:
%%bash
kubectl apply -f simple_cronjob.yaml

cronjob.batch/batch-cronjob created


In [50]:
%%bash
kubectl get cronjobs

NAME            SCHEDULE    SUSPEND   ACTIVE   LAST SCHEDULE   AGE
batch-cronjob   * * * * *   False     0        39s             4m23s


In [51]:
%%bash
kubectl get pods

NAME                             READY   STATUS      RESTARTS   AGE
batch-cronjob-1593796620-bpqms   0/1     Completed   0          2m33s
batch-cronjob-1593796680-8qmm9   0/1     Completed   0          93s
batch-cronjob-1593796740-q44gr   0/1     Completed   0          32s


In [52]:
%%bash
kubectl logs batch-cronjob-1593796740-q44gr

Batch Job started
Batch Job completed successfully


In [53]:
%%bash
kubectl delete cronjob batch-cronjob

cronjob.batch "batch-cronjob" deleted
